In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# The import statements have changed - now need to import each part of the RATE calculation separately
from rate.models import BnnBinaryClassifier # The BNN model
from rate.projections import CovarianceProjection # The projection operator (can also use Pseudoinverse projection)
from rate.cross_validation import cross_validate_bnn # To cross-validate a BNN
from rate.importance import RATE2 # To calculate RATE values


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



# Summary of changes

I have rewritten lots of the code (and will push it to Lorin's branch once we submit).

Most of the code is documented so look there first if you have any problems, but email me if something is not working. I have tried to put argument checks in to help explain the most common errors.

### The BNN models

The main changes are that the BNN models have been coded to match the scikit-learn API, so they have methods like
* fit
* predict
* predict_proba
* score

None of the model functions will work until fit is called (that is when the model actually gets constructed).

Another change is that the logit posterior calculation is now a method of the BNN model itself, so you calculate it using something like

`bnn = BnnBinaryClassifier()`

`bnn.fit(X_train, y_train)`

`M_F, V_F = bnn.logit_posterior(X_test)`

All these functions are documented in `models.py`.

### RATE calculation

This is now done by the `RATE2` function in `importance.py`. Pass it the logit posterior that you will already have comptued using the class method for the bnn. 

This isn't the `RATE_V2` function you were working on a while ago, I just called it this to distinguish it from another older function in the Python code.

The projections are also implemented as their own objects: `CovarianceProjection` and `PseudoinverseProjection`.

They are defined in `projections.py` and inherit from the `ProjectionBase` abstract class. You can define new projections - they just need a method called `esa_posterior` that calculates the effect size analogue posterior from a set of examples and a logit posteiror. This method is called internally by `RATE2` and it uses the `CovarianceProjection` by default.

### Mimic models

I have added a couple of arguments to `train_mimic` and changed how it works internally.

The arguments are now 

`train_mimic(mimic_model, bnn, x_train, y_train=None, x_test=None, n_mc_samples=100, return_time=False)`

The `mimic_model` can be any sklearn method that implements `fit`, so this includes any estimator (e.g. `RandomForestRegressor`) but also cross-validation objects (e.g. `RandomizedSearchCV`), meaning that you can
easily use a pre-specified mimic model or perform cross-validation. The returned mimic model is of the same type 
as the `mimic_model` passed as an argument (e.g. if you pass a `RandomizedSearchCV` you will get one back and then have to extract the model itsef from the `RandomizedSearchCV` object).

The `y_train` argument is useful if you want to train several mimic models on the same predictions - by default it is `None` but if you pass some labels the `bnn` will be ignored and they will be used to train the mimic model. Otherwise (if `y_train` is `None`) the `bnn` is used to generate the predictions.

Finally, whatever estimator/cross-validation object you choose has to be for regression - this is becuase the mimic models are trained on the prediction probabilities (not the class labels) as they contain more information.

### Cross validation

I added a function that does stratified K-fold cross validation for the BNN. Pass it a list of layers, `X`, `y` and the number of folds and it will do cross-validation and return the best model. The type of model (regression or classification) is inferred from the type of labels in `y`.

There are other arguments where you pass 'keyword' : 'value' dictionaries to the BNN __init__, fit and score methods.

### Logging

If you put

`import logging
logging.getLogger('rate').setLevel('INFO')`

at the start of your script the logger should tell you useful information about the computation. It is not fully working yet - it often prints the same things multiple times - but it should stil be useful. You could also use `setLevel('DEBUG')` if you want to have loads of information of dubious relevance printed.

There is also a progress bar for the RATE calculation but I haven't managed to get it to work properly yet either - sometimes it behaves a bit strangely and I don't know why - so if you notice any strange behaviour please let me know!

# Example Code

The code below demonstrates all of this.

Let me know if any of it doesn't work/isn't clear!

### Running the local methods

For the local methods you need a standard neural network that is equivalent to the Bayesian neural network we normally use for RATE.

The function `get_deterministic_nn` does this.

In [2]:
def get_deterministic_nn(bnn):
    """Returns a deterministic neural network with the same architecture as 
    a given Bayesian neural network. Binary classification only"""
    if not isinstance(bnn, BnnBinaryClassifier):
        raise ValueError("get_deterministic_nn is only for BnnBinaryClassifiers")
    nn = Sequential()
    for layer in bnn._logit_model.layers[:-1]:
        deepcopied_layer = layer.__class__.from_config(layer.get_config())
        if hasattr(deepcopied_layer, 'kernel_initializer'):
            deepcopied_layer.build(layer.input_shape)
            deepcopied_layer.kernel.initializer.run(session=K.get_session()) # To reinitialise the weights
        nn.add(deepcopied_layer)
    nn.add(Dense(bnn._logit_model.layers[-1].output_shape[1]))
    nn.add(tf.keras.layers.Activation(activation="sigmoid"))
    nn.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])
    return nn

In [7]:
#
# Load packages
#
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from rate.models import BnnBinaryClassifier
from rate.projections import CovarianceProjection
from rate.mimic import train_mimic
from rate.importance import RATE2

from deepexplain.tensorflow import DeepExplain # Get this from https://github.com/marcoancona/DeepExplain

import logging
logging.getLogger('rate').setLevel('INFO') # Change to 'DEBUG' for a lot more messages

In [4]:
#
# Simulate the data and labels (linearly)
#
n = 1000
p = 3
p_decoy = 7
test_size = 0.3 # Size of test set
n_mc_samples = 10

seed = 123
np.random.seed(seed)

#
# Cross-validation settings for mimic models
#
n_search_iter = 5
k = 3
n_jobs = 7

# Random forest CV grid
rf_param_grid = {
    'n_estimators' : np.arange(10, 1000, 10),
    'max_depth' : np.arange(1, p + p_decoy),
    'max_features' : ['auto', 'log2', 'sqrt']
}

# Gradient boosting machine CV grid
gbm_param_grid = {
    'learning_rate' : [10.0**x for x in [-3.0, -2.0, -1.0]],
    'subsample' : [0.5, 0.7, 0.9, 1.0],
    'n_estimators' : np.arange(10, 1000, 10),
    'max_depth' : np.arange(1, p + p_decoy),
    'max_features' : ['auto', 'log2', 'sqrt']
}

#
# Simulation starts here
#
X = np.random.randn(n, p)

effect_sizes = np.random.randn(X.shape[1])
f = np.dot(X, effect_sizes)
y = (f > np.quantile(f, np.random.uniform(0.4, 0.6))).astype(int)[:,np.newaxis] # Class imbalance will be at worst 60/40

# Add decoy variables
X = np.hstack([X, np.random.randn(n, p_decoy)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

print("Number of class 1: {},\tClass 2:{}".format(y.shape[0]-np.sum(y), np.sum(y))) # Don't want too much of a class imbalance

#
# Fit the models and evaluate their test accuracy
#

# BNN
bnn = BnnBinaryClassifier(verbose=0, n_mc_samples=n_mc_samples)
bnn.fit(X_train, y_train, epochs=10) # Pass kwargs to control network training
print("BNN test score is", bnn.score(X_test, y_test, metric="accuracy"))

nn = get_deterministic_nn(bnn)
nn.fit(X_train, y_train, verbose=0, epochs=10) # Pass kwargs to control network training
print("NN test score is", nn.evaluate(X_test, y_test)[1])

#
# Variable importance
#

# Mimic models (using cross validation)
bnn_soft_predictions = bnn.predict_proba(X_train)
rf_mimic, rf_mimic_time = train_mimic(
    RandomizedSearchCV(
        RandomForestRegressor(),
        rf_param_grid,
        n_iter=n_search_iter,
        cv=k,
        n_jobs=n_jobs),
    bnn, X_train, bnn_soft_predictions, X_test, n_mc_samples, True
)

# The same with no cross-validation
rf_mimic, rf_mimic_time = train_mimic(
            RandomForestRegressor(), bnn, X_train, bnn_soft_predictions, X_test, n_mc_samples, True
)

gbm_mimic, gbm_mimic_time = train_mimic(
    RandomizedSearchCV(
        GradientBoostingRegressor(),
        gbm_param_grid,
        n_iter=n_search_iter,
        cv=k,
        n_jobs=n_jobs),
    bnn, X_train, bnn_soft_predictions, X_test, n_mc_samples, True
)

# RATE
M_F, V_F = bnn.logit_posterior(X_test)
rate_vals, rate_time = RATE2(X_test, M_F, V_F, projection=CovarianceProjection(), return_time=True)

# Saliency-style maps, averaged over examples to give global importance. Average is of absolute value
# This just calculates them and doesn't do anything with them
with DeepExplain(session=K.get_session()) as de:
    input_tensor = nn.layers[0].input
    target_tensor = Model(inputs=input_tensor, outputs=nn.layers[-2].output)(input_tensor)

    for attr_method in ["grad*input", "saliency", "intgrad", "elrp", "occlusion"]:
        imp_vals = de.explain(
            attr_method,
            target_tensor, input_tensor,
            X_test, ys=y_test, # This needs 2D arrays so you may have to add an axis here if using 1D labels
            batch_size=64)

Number of class 1: 592,	Class 2:408
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


BNN test score is 0.9633333333333334
300/300 [==============================] - 0s 164us/sample - loss: 0.1924 - acc: 0.9600


NN test score is 0.96


INFO:rate.mimic:Mimic R^2 on x_test: 0.946
/home/jonathan/anaconda3/envs/rate-bnn/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
INFO:rate.mimic:Mimic R^2 on x_test: 0.926
INFO:rate.mimic:Mimic R^2 on x_test: 0.946
INFO:rate.importance:Calculating RATE values for 1 classes, 300 examples and 10 variables
INFO:rate.importance:Calculating RATE values for class 1 of 1
100%|██████████| 10/10 [00:00<00:00, 2704.43it/s]
INFO:rate.importance:The RATE calculation took 0.008 seconds


Calculating RATE values for 1 classes, 300 examples and 10 variables
Calculating RATE values for class 1 of 1
The RATE calculation took 0.008 seconds
